In [2]:
import requests
import pandas as pd
# from htmlmerger import HtmlMerger
import os
import datetime

In [3]:
url = "https://api.usaspending.gov"
df_opps = pd.read_xml('./GrantsDBExtract20230910v2.xml')

In [4]:
cfda_urls = pd.read_csv('./AssistanceListings_GrantsGov_PUBLIC_DAILY_20230910.csv', encoding = "ISO-8859-1")

In [5]:
cfda_urls.loc[cfda_urls['Program Number'] == 10.001]['URL'][0]

'https://sam.gov/fal/8138603977434d65a63073a7e5f5cc13/view'

In [6]:
df_opps['PostDate'] = pd.to_datetime(df_opps['PostDate'], format='%m%d%Y')
df_opps['CloseDate'] = pd.to_datetime(df_opps['CloseDate'], format='%m%d%Y')
df_opps = df_opps[df_opps['CloseDate'] > pd.to_datetime("2023-09-01")]
searchfor = ['RFI']
df_opps = df_opps[~df_opps["OpportunityNumber"].str.contains('|'.join(searchfor))]
df_opps = df_opps[df_opps['CFDANumbers'].notna()]
df_opps = df_opps.astype({"CFDANumbers":'float'})

remove_nones = ["Description", "AgencyName", "AdditionalInformationOnEligibility"]
for column in remove_nones:
    blank_list = []
    for i in df_opps[column]:
        if type(i) == str:
            blank_list.append(i)
        else:
            blank_list.append("No " + column)
    df_opps[column] = blank_list

In [7]:
popl_list = [
                {
                    "country": 'BGD'},
                {
                    "country": "BOL"},
                {
                    "country": "BDI"},
                {
                    "country": "KHM"},
                {
                    "country": "COD"},
                {
                    "country": "DOM"},
                {
                    "country": "ETH"},
                {
                    "country": "GTM"},
                {
                    "country": "HTI"},
                {
                    "country": "IDN"},
                {
                    "country": "KEN"},
                {
                    "country": "MOZ"},
                {
                    "country": "NIC"},
                {
                    "country": "PER"}, 
                {
                    "country": "PHL"},
                {
                    "country": "RWA"},
                {
                    "country": "SSD"},
                {
                    "country": "UGA"},
                ]

country_names = {
    "BGD" : ["Bangladesh", "Dhaka", "Asia", "Pacific", "Bengali", "Bangladeshi"],
    "BOL" : ["Bolivia", "La Paz", "Sucre", "Latin", "Central America", "South America", "Caribbean", "America", "Bolivian"],
    "BDI" : ["Burundi", "Gitega", "Africa", "Burundian"],
    "KHM" : ["Cambodia","Phnom Penh", "Asia", "Pacific", "Cambodian"],
    "DOM" : ["Dominican Republic", "Santo Domingo", "Latin", "Central America", "South America", "Caribbean", "America", "Dominican"],
    "COD" : ["Congo", "Kinshasa", "Africa", "Congolese"],
    "ETH" : ["Ethiopia","Addis Ababa", "Africa", "Ethiopian"],
    "GTM" : ["Guatemala", "Guatemala City", "Latin", "Central America", "South America", "Caribbean", "America", "Guatemalan"],
    "HTI" : ["Haiti", "Port au Prince", "Latin", "Central America", "South America", "Caribbean", "America", "Haitian"],
    "IDN" : ["Indonesia", "Jakarta", "Asia", "Island", "Pacific", "Indonesian"],
    "KEN" : ["Kenya", "Nairobi", "Africa", "Kenyan"],
    "MOZ" : ["Mozambique", "Maputo", "Africa", "Mozambican"],
    "NIC" : ["Nicaragua", "Managua", "Latin", "Central America", "South America", "Caribbean", "America", "Nicaraguan"],
    "PER" : ["Peru", "Lima", "Latin", "Central America", "South America", "Caribbean", "America", "Peruvian"],
    "PHL" : ["Philippines", "Manila", "Asia", "Island", "Pacific", "Filipino"],
    "RWA" : ["Rwanda", "Kigali", "Africa", "Tutsi", "Hutu", "Rwandan"],
    "SSD" : ["Sudan", "Juba", "Africa", "Sudanese"],
    "UGA" : ["Uganda", "Kampala", "Africa", "Ugandan"]
}

date_list = {
    "FY19" : ["2018-10-01", "2019-09-30"], 
    "FY20" : ["2019-10-01", "2020-09-30"], 
    "FY21" : ["2020-10-01", "2021-09-30"], 
    "FY22" : ["2021-10-01", "2022-09-30"], 
    "FY23" : ["2022-10-01", "2023-09-30"]
    }

countries = open("countries.txt", "r")
countries = countries.read()
countries = countries.split("\n")

In [8]:
html_body_start = """<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
.collapsible {
  background-color: #777;
  color: white;
  cursor: pointer;
  padding: 18px;
  width: 100%;
  border: none;
  text-align: left;
  outline: none;
  font-size: 15px;
}

.active, .collapsible:hover {
  background-color: #555;
}

.content {
  padding: 0 18px;
  max-height: 0;
  overflow: hidden;
  transition: max-height 0.2s ease-out;
  background-color: #f1f1f1;
}
</style>
</head>
<body>""" #<div style='display: flex; align-items: center; justify-content: center; flex-direction: column;>'

html_body_end = """<script>
var coll = document.getElementsByClassName("collapsible");
var i;

for (i = 0; i < coll.length; i++) {
  coll[i].addEventListener("click", function() {
    this.classList.toggle("active");
    var content = this.nextElementSibling;
    if (content.style.maxHeight){
      content.style.maxHeight = null;
    } else {
      content.style.maxHeight = content.scrollHeight + "px";
    } 
  });
}
</script>

</body>"""

In [9]:
for country in popl_list:
    os.mkdir(country_names[country['country']][0])

In [11]:
final_df = pd.DataFrame()
final_df_cfda = pd.DataFrame()
for country in popl_list:
    print(country)

    # try:
    #     os.mkdir(country_names[country['country']][0])
    #     os.mkdir(country_names[country['country']][0] + "/html")
    # except:
    #     continue 

    country_df = pd.DataFrame()
    endpoint_cfda = "/api/v2/search/spending_by_category/cfda/"
    payload_cfda = {
        "limit": 100,
        "page": 1,
        "filters": {
            "award_type_codes": ["02","03","04","05"],
            "time_period": [{"start_date": "2018-10-01", "end_date": "2023-09-30"}],
            "place_of_performance_locations": [country]
        }
    }
    response_cfda = requests.post(f"{url}{endpoint_cfda}", json=payload_cfda)
    data_cfda = response_cfda.json()
    df_cfda = pd.DataFrame(data_cfda["results"])
    df_cfda = df_cfda[df_cfda.amount >= 250_000]

    cfda_start_html = """<div style="display: flex; align-items: center; justify-content: center; flex-direction: column;">
            <h1>""" + country_names[country['country']][0] + """ USG Breakdown</h1>
            <p style="style="font-style: italic;">USG funding per CFDA category for FY18 through FY23</p>
            <table border="1" class="dataframe">
            <thead>
                <tr style="text-align: center;">
                <th></th>
                <th>CFDA</th>
                <th>Name</th>
                <th>Amount (USD)</th>
                </tr>
            </thead>
            <tbody>"""
    
    for index, row in df_cfda.iterrows():
        sam_url = cfda_urls.loc[cfda_urls['Program Number'] == float(row['code'])]['URL'].item()
        cfda_start_html += "<tr><th>" + str(index) + "</th><td><a href='" + sam_url + "' target='_blank'>" + str(row['code']) + "</a></td><td><a href='#" + str(row['code']) + "'>" + row['name'] + "</a></td><td>" + '${:,.2f}'.format(int(row['amount'])) +"</td></tr>"

    cfda_start_html += "</tbody></table></div><br>"

    # with open("./" + country_names[country['country']][0] + "/html/y.html", "w") as file:
    #     file.write(cfda_start_html)

    # if final_df_cfda.empty:
    #     final_df_cfda = df_cfda
    #     df_cfda.to_csv("./" + country_names[country['country']][0] + "/cfda.csv")
    # else:
    #     final_df_cfda = pd.concat([final_df_cfda, df_cfda], ignore_index = True)

    for key in date_list:
        print(key)
        df_full = True
        page_index = 1
        while df_full:
            print(page_index)
            endpoint_award = "/api/v2/search/spending_by_award/"
            payload_award = {
                "limit": 100,
                "page": page_index,
                "filters": {
                    "award_type_codes": ["02","03","04","05"],
                    "time_period": [{"start_date": date_list[key][0], "end_date": date_list[key][1]}],
                    "place_of_performance_locations": [country],
                    "program_numbers": df_cfda.code.to_list()
                },
                "fields": [
                    "Award ID",
                    "Recipient Name",
                    "Start Date",
                    "End Date",
                    "Award Amount",
                    "Awarding Agency",
                    "Awarding Sub Agency",
                    "Contract Award Type",
                    "Award Type",
                    "Funding Agency",
                    "Funding Sub Agency",
                    "CFDA Number",
                    "Description"
                ]
            }
            response_award = requests.post(f"{url}{endpoint_award}", json=payload_award)
            data_award = response_award.json()
            df_award = pd.DataFrame(data_award["results"])
            if df_award.empty:
                df_full = False
            df_award["FY"] = key
            df_award["Country"] = country['country']
            if country_df.empty:
                country_df = df_award
            else:
                country_df = pd.concat([country_df, df_award], ignore_index = True)
            page_index += 1

    country_df = country_df.astype({"CFDA Number":'float'})
    country_df = country_df.drop_duplicates(subset='Award ID', keep="first")
    country_df['End Date'] = pd.to_datetime(country_df['End Date']).dt.date
    country_df = country_df.sort_values(by='End Date')

    countries_copy = countries.copy()
    for related_names in country_names[country['country']]:
        print(related_names)
        countries_copy.remove(related_names)
    df_opps_copy = df_opps[
        ~df_opps["OpportunityTitle"].str.contains('|'.join(countries_copy)) 
        & ~df_opps["Description"].str.contains('|'.join(countries_copy)) 
        & ~df_opps["AgencyName"].str.contains('|'.join(countries_copy))
        & ~df_opps["AdditionalInformationOnEligibility"].str.contains('|'.join(countries_copy))]
    df_opps_copy = df_opps_copy[(df_opps_copy['CFDANumbers'].isin([float(i) for i in country_df['CFDA Number'].unique()]))]
    # df_opps_copy.to_csv("./" + country_names[country['country']][0] + "/opps.csv")

    # with open("./" + country_names[country['country']][0] + "/html/z.html", "w") as file:
    #     file.write(html_body)

    html_body_final = html_body_start
    for cfda, name, amount in zip(df_cfda['code'], df_cfda['name'], df_cfda['amount']):
        sam_url = cfda_urls.loc[cfda_urls['Program Number'] == float(cfda)]['URL'].item()
        html_body_final += "<hr><a href='" + sam_url + "'><h2 id='"+ str(cfda) + "'>" + str(cfda) + ": " + str(name) + "</h2></a><p>Amount provided from FY218 through FY23 <b>" + '${:,.2f}'.format(int(amount)) + "</b></p>"

        for index, row in df_opps_copy[df_opps_copy['CFDANumbers'] == float(cfda)].iterrows():
            print(index)
            html_body_final += "<button class='collapsible'><b>[Posted in " + row['PostDate'].strftime('%Y') + "]  </b> " + str(row['OpportunityTitle']) + "</button>"
            html_body_final += "<div class='content'><p>" + str(row['Description']) + "<br><br>" + "<a href='https://www.grants.gov/web/grants/view-opportunity.html?oppId=" + str(row['OpportunityID']) + "' target='_blank'>Grants.gov Link</a></p></div>"

    html_body_final += html_body_end

    with open("./" + country_names[country['country']][0] + "/open_opps_report.html", "w") as file:
        file.write(cfda_start_html + html_body_final)

    html_body_final = html_body_start
    for cfda, name, amount in zip(df_cfda['code'], df_cfda['name'], df_cfda['amount']):
        sam_url = cfda_urls.loc[cfda_urls['Program Number'] == float(cfda)]['URL'].item()
        html_body_final += "<hr><a href='" + sam_url + "'><h2 id='"+ str(cfda) + "'>" + str(cfda) + ": " + str(name) + "</h2></a><p>Amount provided from FY218 through FY23 <b>" + '${:,.2f}'.format(int(amount)) + "</b></p>"

        for index, row in country_df[country_df['CFDA Number'] == float(cfda)].iterrows():
            print(index)
            html_body_final += "<button class='collapsible'><b>[Ending in " + str(row['End Date'].year) + "]  </b> " + str(row['Award ID']) + " " + str(row['Recipient Name']) + "</button>"
            html_body_final += "<div class='content'><p>" + '${:,.2f}'.format(int(row['Award Amount'])) + " LOA from " + str(row['Start Date']) + " to " + str(row['End Date']) + "<br>TYPE: " + str(row['Award Type']) + "<br><br>" + str(row['Description']) + "<br><br>" +"<a href='https://www.usaspending.gov/award/" + str(row['generated_internal_id']) + "' target='_blank'>USAspending.gov Link</a></p></div>"

    html_body_final += html_body_end

    with open("./" + country_names[country['country']][0] + "/awards_report.html", "w", encoding="utf-8") as file:
        file.write(cfda_start_html + html_body_final)

#     """
# <p>Collapsible Set:</p>
# <button class="collapsible">Open Section 1</button>
# <div class="content">
#   <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
# </div>
# <button class="collapsible">Open Section 2</button>
# <div class="content">
#   <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
# </div>
# <button class="collapsible">Open Section 3</button>
# <div class="content">
#   <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
# </div>"""
    
    # if final_df.empty:
    #     final_df = country_df
    #     country_df.to_csv("./" + country_names[country['country']][0] + "/awards.csv")
    #     break
    # else:
    #     final_df = pd.concat([final_df, country_df], ignore_index = True)

    # merger = HtmlMerger(input_directory="./" + country_names[country['country']][0] + "/html/", output_file="./" + country_names[country['country']][0] + "/html/merged.html")
    # merger.merge(clean=True)

{'country': 'BGD'}
FY19
1
2
3
FY20
1
2
3
FY21
1
2
3
FY22
1
2
3
FY23
1
2
3
Bangladesh
Dhaka
Asia
Pacific
Bengali
Bangladeshi
33288
35939
36027
46071
49942
54044
61552
47507
54525
59775
59649
53245
53246
56137
58807
58748
133
96
127
132
131
126
99
109
128
102
122
106
124
81
82
83
79
125
129
130
95
76
85
87
108
118
78
104
98
103
105
146
120
121
110
100
97
84
94
152
77
147
140
294
73
75
153
281
137
299
134
86
123
284
155
119
143
74
283
150
151
142
80
274
117
478
116
148
112
282
646
476
145
114
141
647
479
477
473
144
107
475
303
474
301
637
159
634
163
636
300
111
154
780
101
157
767
632
302
635
316
788
315
633
480
149
507
766
768
113
482
483
298
639
648
649
158
651
161
295
296
297
765
764
770
676
645
769
481
638
115
487
485
484
162
486
93
650
631
786
789
164
785
784
783
782
781
156
754
414
70
219
174
520
519
521
522
691
690
172
173
332
333
331
138
135
88
90
91
92
89
1
7
8
330
518
687
139
160
136
689
55
59
41
51
22
52
11
188
34
183
47
48
60
44
42
195
38
544
179
178
542
707
706
705
703
704


In [20]:
country_df

,internal_id,Award ID,Recipient Name,Start Date,End Date,Award Amount,Awarding Agency,Awarding Sub Agency,Contract Award Type,Award Type,Funding Agency,Funding Sub Agency,CFDA Number,Description,awarding_agency_id,agency_slug,generated_internal_id,FY,Country
300,48335272.0,AID617A000800008,THE AIDS SUPPORT ORGANIZATION (TASO),2007-12-13,2009-06-30,4460357.0,Agency for International Development,Agency for International Development,None,COOPERATIVE AGREEMENT (B),Agency for International Development,Agency for International Development,98.001,TO SUPPORT DELIVERY OF HIV/AIDS PREVENTION PA...,801.0,agency-for-international-development,ASST_NON_AID617A000800008_7200,FY19,UGA
253,48335360.0,AID617LA1000002,CONSORTIUM FOR ELECTIONS & POLITICAL PROCESS S...,2010-05-06,2011-08-30,1105605.0,Agency for International Development,Agency for International Development,None,COOPERATIVE AGREEMENT (B),Agency for International Development,Agency for International Development,98.001,SUPPORT TO ELECTORAL COMMISSION FOR THE NATION...,801.0,agency-for-international-development,ASST_NON_AID617LA1000002_7200,FY19,UGA
301,48335271.0,AID617A000800006,UNIVERSITY RESEARCH CO LLC,2007-12-12,2011-08-31,5114846.0,Agency for International Development,Agency for International Development,None,COOPERATIVE AGREEMENT (B),Agency for International Development,Agency for International Development,98.001,TO SUPPORT FOOD AND NUTRITION INTERVENTIONS FO...,801.0,agency-for-international-development,ASST_NON_AID617A000800006_7200,FY19,UGA
302,48335269.0,AID617A000800002,NATIONAL DEMOCRATIC INSTITUTE FOR INTERNATIONA...,2007-12-12,2011-12-11,6008538.0,Agency for International Development,Agency for International Development,None,COOPERATIVE AGREEMENT (B),Agency for International Development,Agency for International Development,98.001,TO PROVIDE SUPPORT FOR A PROGRAM IN SUPPORT OF...,801.0,agency-for-international-development,ASST_NON_AID617A000800002_7200,FY19,UGA
303,48335263.0,AID617A000600009,JSI RESEARCH & TRAINING INSTITUTE INC,2006-08-15,2012-06-15,17633850.0,Agency for International Development,Agency for International Development,None,COOPERATIVE AGREEMENT (B),Agency for International Development,Agency for International Development,98.001,"HIV/AIDS, TUBERCULOSIS AND MALARIA SERVICES IN...",801.0,agency-for-international-development,ASST_NON_AID617A000600009_7200,FY19,UGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,160937135.0,K43TW012637,THE INFECTIOUS DISEASES INSTITUTE LIMITED,2023-07-15,2028-06-30,109911.0,Department of Health and Human Services,National Institutes of Health,None,PROJECT GRANT (B),Department of Health and Human Services,National Institutes of Health,93.989,PEER DELIVERED HIV/SYPHILIS SELF-TESTING WITH ...,806.0,department-of-health-and-human-services,ASST_NON_K43TW012637_7529,FY23,UGA
1465,161260131.0,R01NS129041,MAKERERE UNIVERSITY COLLEGE O F HEALTH SCIENCES,2023-08-01,2028-07-31,390769.0,Department of Health and Human Services,National Institutes of Health,None,PROJECT GRANT (B),Department of Health and Human Services,National Institutes of Health,93.853,SELF-MANAGEMENT INTERVENTION FOR REDUCING EPIL...,806.0,department-of-health-and-human-services,ASST_NON_R01NS129041_7529,FY23,UGA
1467,162607296.0,R01HD111692,MBARARA UNIVERSITY OF SCIENCE AND TECHNOLOGY,2023-09-05,2028-08-31,488413.0,Department of Health and Human Services,National Institutes of Health,None,PROJECT GRANT (B),Department of Health and Human Services,National Institutes of Health,93.865,INTEGRATION OF A PATIENT-CENTERED MOBILE HEALT...,806.0,department-of-health-and-human-services,ASST_NON_R01HD111692_7529,FY23,UGA
1480,162605064.0,NU2GGH002493,AFRICAN FIELD EPIDEMIOLOGY NETWORK LTD,2023-09-30,2028-09-29,3250000.0,Department of Health and Human Services,Centers for Disease Control and Prevention,None,COOPERATIVE AGREEMENT (B),Department of Health and Human Services,Centers for Disease Control and Prevention,93.067,STRENGTHENING LABORAT

In [34]:
endpoint_acc = "/api/v2/search/spending_by_category/federal_account/"
payload_acc = {
    "limit": 100,
    "page": 1,
    "filters": {
        "award_type_codes": ["02","03","04","05"],
        "time_period": [{"start_date": "2018-10-01", "end_date": "2023-09-30"}],
        "place_of_performance_locations": [{"country": 'BGD'}],
        "award_ids": ["AID388A000900123"]
    }
}
response_acc = requests.post(f"{url}{endpoint_acc}", json=payload_acc)
data_acc = response_acc.json()
df_acc = pd.DataFrame(data_acc["results"])

In [35]:
df_acc

,amount,id,code,name
0,-181651.0,4240,019-1031,"Global Health Programs, State"
